In [1]:
import numpy as np
from collections import defaultdict
import itertools

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

seed = 514
np.random.seed(seed)

In [2]:
# select targets
categories = [ 'talk.religion.misc', 'comp.graphics', 'sci.space']
K = len(categories)

In [3]:
# load data
newsgroups = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=seed)
N = len(newsgroups.data)

In [4]:
# create vectorizer
vectorizer = CountVectorizer(stop_words='english')
tdata = vectorizer.fit_transform(newsgroups.data, )
V = len(vectorizer.vocabulary_)

In [5]:
# prepare words
wordLst = defaultdict(list)

idxLst = zip(*tdata.nonzero())
for idx in idxLst:
    wordLst[idx[0]].append((idx[1], tdata[idx]))

In [6]:
# given a, b
a = 0.1
b = 1.0

# initialize pZ
pZ = np.zeros((N, K))
for d in range(N):
    for k in range(K):
        pZ[d,k] = 1.0 / K

# initialize Z
np.random.seed(514)
Z = np.zeros((N,), dtype=np.int)
for d in range(N):
    c = np.random.choice(range(K), 1, p=pZ[d])[0]
    Z[d] = c

In [7]:
# main process
for i in xrange(200):
    if i % 10 == 0:
        print 'Now is working on Iteration #%d...' % i
    
    np.random.seed(514)

    # update pP
    sD = np.zeros((K,))
    for d in range(N):
        sD[Z[d]] += 1
    
    pP = np.random.dirichlet(sD + b, 1)[0]

    # update pT
    sW = np.zeros((K, V))
    for d, vcLst in wordLst.iteritems():
        for v, c in vcLst:
            sW[Z[d], v] += c

    pT = np.zeros((K, V))
    for k in range(K):
        pT[k] = np.random.dirichlet(sW[k] + a, 1)[0]

    # update pZ
    for d in range(N):
        for k in range(K):
            pZ[d,k] = np.log(pP[k])
            for v, c in wordLst[d]:
                pZ[d,k] += c * np.log(pT[k,v])

        # trick
        maxv = max(pZ[d])
        pZ[d] = np.exp(pZ[d] - maxv)
        pZ[d] /= sum(pZ[d])
    
    # update Z
    for d in range(N):
        c = np.random.choice(range(K), 1, p=pZ[d])[0]
        Z[d] = c

Now is working on Iteration #0...
Now is working on Iteration #10...
Now is working on Iteration #20...
Now is working on Iteration #30...
Now is working on Iteration #40...
Now is working on Iteration #50...
Now is working on Iteration #60...
Now is working on Iteration #70...
Now is working on Iteration #80...
Now is working on Iteration #90...
Now is working on Iteration #100...
Now is working on Iteration #110...
Now is working on Iteration #120...
Now is working on Iteration #130...
Now is working on Iteration #140...
Now is working on Iteration #150...
Now is working on Iteration #160...
Now is working on Iteration #170...
Now is working on Iteration #180...
Now is working on Iteration #190...
CPU times: user 14min 35s, sys: 13.5 s, total: 14min 49s
Wall time: 16min 9s


In [8]:
# maximize prediction
print pP
argm = np.argmax(pZ, axis=1)

def calACC(perm):
    pred = []
    for v in argm:
        pred.append(perm[v])
    return accuracy_score(newsgroups.target, pred)

maxc = 0
for perm in itertools.permutations(range(K), K):
    pred = []
    for v in argm:
        pred.append(perm[v])
    acc = accuracy_score(newsgroups.target, pred)
    if acc > maxc:
        maxc = acc
        maxp = pred

[ 0.35005771  0.25799032  0.39195198]
0.396058732612
Truth [2 1 1 0 0 0 0 0 0 1 1 1 0 1 0 2 1 0 2 2 1 0 1 0 1 0 1 1 2 0 1 1 0 0 0 0 0
 2 1 1 1 0 0 0 1 1 0 0 1 1 1 0 2 0 1 0 0 0 0 2 0 1 1 0 2 0 0 0 2 0 2 0 0 1
 2 2 2 1 1 0 0 1 2 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1]
Prediction [1 1 2 2 2 2 1 0 0 1 1 0 2 1 0 2 2 0 2 1 1 0 2 2 2 2 1 2 2 0 0 1 1 1 1 0 1
 1 1 2 2 2 2 1 2 2 2 2 2 0 1 2 0 2 1 1 1 2 2 1 2 0 1 0 1 1 1 2 1 2 1 0 0 1
 0 1 2 1 1 2 0 2 0 2 1 2 2 1 2 2 2 2 2 1 1 2 0 2 0 0]


In [9]:
# ten most frequent words
words = vectorizer.get_feature_names()
for k in range(K):
    sortLst = sorted(list(enumerate(pT[k])), key=lambda x: x[1], reverse=True)
    print [words[p[0]] for p in sortLst[:10]]

[u'edu', u'lines', u'subject', u'organization', u'com', u'space', u'writes', u'article', u'posting', u'just']
[u'edu', u'image', u'jpeg', u'com', u'subject', u'organization', u'lines', u'space', u'graphics', u'software']
[u'edu', u'lines', u'subject', u'organization', u'com', u'space', u'writes', u'article', u'like', u'posting']
